In [1]:
import os
import h5py
import numpy as np
from tqdm import tqdm

# 配置路径
src_dir = "/data3/wyf/yiyuan_npy_osa"
dst_dir = "/data3/wyf/yiyuan_npy_osa_h5"

# 确保目标目录存在
os.makedirs(dst_dir, exist_ok=True)

# 遍历源目录中的每个子文件夹
for subdir_name in tqdm(os.listdir(src_dir), desc="处理子文件夹"):
    subdir_path = os.path.join(src_dir, subdir_name)
    
    # 跳过非目录文件
    if not os.path.isdir(subdir_path):
        continue
        
    # 创建同名的h5文件
    h5_filename = os.path.join(dst_dir, f"{subdir_name}.h5")
    
    # 删除已存在的h5文件（避免旧数据干扰）
    if os.path.exists(h5_filename):
        os.remove(h5_filename)
    
    # 收集子文件夹中的所有npy文件
    npy_files = sorted([f for f in os.listdir(subdir_path) if f.endswith('.npy')])
    
    with h5py.File(h5_filename, 'w') as h5f:
        # 处理每个npy文件
        for npy_file in tqdm(npy_files, desc=f"打包 {subdir_name}", leave=False):
            file_path = os.path.join(subdir_path, npy_file)
            data = np.load(file_path)
            
            # 使用无扩展名的文件名作为数据集名
            dataset_name = os.path.splitext(npy_file)[0]
            
            # 创建数据集
            h5f.create_dataset(
                name=dataset_name,
                data=data,
                # 使用gzip压缩减少存储空间
                compression="gzip",
                # 设置合适的压缩等级（1-9，9压缩率最高）
                compression_opts=6,
                # 使用分块存储优化大文件访问
                chunks=True,
                # 自动计算类型和形状
                dtype=data.dtype
            )
            
        # 添加来源元数据
        h5f.attrs['source_folder'] = subdir_path
        h5f.attrs['npy_files_count'] = len(npy_files)

print(f"\n处理完成！所有文件已保存至: {dst_dir}")

处理子文件夹: 100%|██████████| 317/317 [37:13<00:00,  7.05s/it]  


处理完成！所有文件已保存至: /data3/wyf/yiyuan_npy_osa_h5
